# Optimal Pokemon GO Battle Team

### The following script will use data from simulations in Pvpoke and meta data from Silphroad (or hand collected data) to find the optimal team

#### This first block will pull in the Pvpoke data which was obtained through tedious simulations

In [1]:
import pandas
import numpy
pathname = "/Users/chugh/Documents/Pokemon Battle/Pokemon matchups.csv"
matchups = pandas.read_csv(pathname, index_col = 0)
matchups = matchups.reindex(sorted(matchups.columns), axis=1)

matchups_poke = numpy.array(matchups.columns)

#### Now we will pull in probability data from either SilphRoad (first block), manually collected data (second block), or no data at all (third block)

##### Silphroad data

In [5]:
#this code will extract pokemon probabilities from silphroad data
pathname = "/Users/chugh/Documents/Pokemon Battle/probs_may.json"
prob_data = pandas.read_json(pathname)

#extract first pokemon data
first_poke = prob_data['lead_poke']
first_prob = prob_data['lead_species_prevelance']

first_poke, idx = numpy.unique(first_poke, return_index = True)
first_prob = first_prob[idx]
first_prob = first_prob.reset_index(drop = True)

#extract second pokemon data
second_poke = prob_data['second_poke']

#extract third pokemon data
third_poke = prob_data['third_poke']

#for second and third pokemon, probability has to be calculated
team_prob = prob_data['team_prevalance']
def prob_calc(team_prob, poke_list):
    poke_unq, idx = numpy.unique(poke_list, return_inverse = True)
    num_poke = len(poke_unq)
    poke_prob = numpy.zeros(num_poke)
    num_team = len(team_prob)
    for curr_poke in range(0,num_team-1):
        curr_prob = team_prob[curr_poke]
        poke_prob[idx[curr_poke]] = poke_prob[idx[curr_poke]] + curr_prob
        
    return poke_prob, poke_unq

second_prob, second_poke = prob_calc(team_prob, second_poke)
third_prob, third_poke = prob_calc(team_prob, third_poke)

#need to move all probabilities over two decimal places
first_prob = first_prob/100
second_prob = second_prob/100
third_prob = third_prob/100
idx = first_prob > 0.015
print(first_poke[idx])
first_prob

['altaria' 'azumarill' 'bastiodon' 'clefable' 'deoxys-defense' 'haunter'
 'hypno' 'meganium' 'melmetal' 'raichu-alola' 'registeel' 'skarmory'
 'swampert' 'toxicroak' 'umbreon' 'venusaur' 'whiscash']


0     0.1004
1     0.0008
2     0.0005
3     0.1197
4     0.0262
       ...  
64    0.0020
65    0.0074
66    0.0467
67    0.0112
68    0.0012
Name: lead_species_prevelance, Length: 69, dtype: float64

##### Manually collected data

In [66]:
#alternative using my collected data
pathname = "/Users/chugh/Documents/Pokemon Battle/Pokemon data new.csv"
prob_pre_data = pandas.read_csv(pathname, index_col = 0)
total_matches = len(prob_pre_data)

def calculate_prob(column_num, data, total_matches):
    pokemon_list = data[column_num]
    pokemon_list = pokemon_list[~(pokemon_list.isnull())] #get rid of nans
    pokemon_unq = numpy.unique(pokemon_list)
    pokemon_prob = numpy.zeros(len(pokemon_unq))
    for curr_poke in pokemon_list:
        loc = numpy.where(pokemon_unq == curr_poke)
        pokemon_prob[loc] += 1

    prob = pokemon_prob/total_matches
    return prob, pokemon_unq

first_prob, first_poke = calculate_prob('first pokemon', prob_pre_data, total_matches)
second_prob, second_poke = calculate_prob('second pokemon', prob_pre_data, total_matches)
third_prob, third_poke = calculate_prob('third pokemon', prob_pre_data, total_matches)

##### Remove influence of probability on outcome

In [704]:
#can also process data agnostic of probability
first_prob = numpy.ones(len(first_prob))
second_prob = numpy.ones(len(second_prob))
third_prob = numpy.ones(len(third_prob))

#### Now we will process the probability data and matchup data to produce the suggested first, second, and third pokemon

In [8]:
#find unique values for pokemon names and probabilities
#matchups_poke will be input for pre_matchups
def best_pokemon(poke_list, prob_list, pre_matchups, pre_matchups_poke, pre_best_poke, pre_best_poke_2):

    #limit pokemon and probabilities to those included in matchups array
    poke_ind = pandas.Index(poke_list)
    poke_match = poke_ind.intersection(pre_matchups_poke)

    #create new pokemon and probability list with exclusions
    i = 0
    prob = []
    poke = []
    for curr_spec in poke_list:
        if curr_spec in poke_match:
            prob.append(prob_list[i])
            poke.append(poke_list[i])
        i += 1
        
    #we can use some sort of weighting factor to make probability more important
    prob = numpy.power(prob,3)
    
    #now limit matches array columns to those included in probability arrays
    poke_match_2 = pre_matchups.columns.intersection(poke)
    matchups = pre_matchups[poke_match_2]
    matchups = matchups.reindex(sorted(matchups.columns), axis=1)
    
    #before next step, need to multiply by weighting factor if any
    mult_fac = []
    if len(pre_best_poke) != 0:
        #find pokemon that has already been selected in matchup array
        best_matchups = matchups.loc[pre_best_poke, :] 
        #if they are good against something, we want to decrease the probability and vice versa
        mult_fac = 1000/best_matchups
        #now multiply probability of second pokemon by multiplication factor
        prob = prob*mult_fac
        
    mult_fac_2 = []
    if len(pre_best_poke_2) != 0:
        #find pokemon that has already been selected in matchup array
        best_matchups_2 = matchups.loc[pre_best_poke_2, :] 
        #if they are good against something, we want to decrease the probability and vice versa
        mult_fac_2 = 1000/best_matchups_2
        #now multiply probability of second pokemon by multiplication factor
        prob = prob*mult_fac_2

    i = 0
    for curr_prob in prob:
        matchups[poke[i]] = matchups[poke[i]]*curr_prob
        i += 1
    
    #now calculate the correct pokemon
    prob_sum = matchups.sum(axis=1)
    prob_sum = prob_sum.to_frame()
    prob_sum = prob_sum.sort_values(0)
    loc = len(prob_sum.index)
    best_poke = prob_sum.index[loc-1]
    
    return best_poke, prob_sum, loc

first_best_poke, first_prob_sum, temp = best_pokemon(first_poke, first_prob, matchups, matchups_poke, [], [])
#first_best_poke = 'raichu-alola'
second_best_poke, second_prob_sum, temp = best_pokemon(second_poke, second_prob, matchups, matchups_poke, first_best_poke, [])
second_best_poke = 'venusaur'
third_best_poke, third_prob_sum, temp = best_pokemon(third_poke, third_prob, matchups, matchups_poke, first_best_poke, second_best_poke)
print("Lead Pokemon: ", first_best_poke)
print("Second Pokemon: ", second_best_poke)
print("Third Pokemon: ", third_best_poke)
third_prob_sum

Lead Pokemon:  raichu-alola
Second Pokemon:  venusaur
Third Pokemon:  whiscash


,0
Pokemon,
altaria,7.928301
tropius,8.196306
vigoroth,8.240094
skarmory,8.468326
bastiodon,8.734880
hypno,8.859611
umbreon,9.022098
marowak-alola,9.053850
deoxys-defense,9.100543


### You can modify the output above to give a list of pokemon from least preferred to most preferred. You can also modilfy the first or second pokemon selection to see how that influences the outcomes. Finally, you can try different weightings of probability and matchups to see what produces the best results.